In [18]:
######working

import cv2
import pytesseract
from pytesseract import Output
import pandas as pd
import numpy as np

# Load the image
image_path = '15.jpeg'
image = cv2.imread(image_path)

# Check if the image was successfully loaded
if image is None:
    print(f"Error: Unable to load image at path {image_path}")
    exit(1)

# Preprocess the image
def preprocess_image(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Sharpen the image
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharp = cv2.filter2D(gray, -1, kernel)
    
    # Denoise the image
    denoised = cv2.fastNlMeansDenoising(sharp, h=30)
    
    # Resize the image
    height, width = denoised.shape
    resized = cv2.resize(denoised, (width * 3, height * 3))
    
    # Apply both adaptive and global thresholding
    adaptive_thresh = cv2.adaptiveThreshold(resized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                            cv2.THRESH_BINARY, 11, 2)
    _, global_thresh = cv2.threshold(resized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Combine both thresholding results
    combined_thresh = cv2.bitwise_or(adaptive_thresh, global_thresh)
    
    return combined_thresh

# Preprocess the image
thresh = preprocess_image(image)

# Use Tesseract to perform OCR with custom configurations
custom_config = r'--oem 3 --psm 6'
data = pytesseract.image_to_data(thresh, output_type=Output.DICT, config=custom_config)

# Extract the OCR results into a DataFrame
df = pd.DataFrame(data)

# Filter out non-table data
table_data = df[df['text'].str.strip().astype(bool)]

# Initialize variables to store row data
rows = []
current_row = []
current_line_num = table_data['line_num'].min()

# Iterate through the table data
for i, row in table_data.iterrows():
    if row['line_num'] != current_line_num:
        if current_row:
            rows.append(' '.join(current_row))
        current_row = []
        current_line_num = row['line_num']
    
    # Concatenate text within the same line
    current_row.append(row['text'])

# Append the last row
if current_row:
    rows.append(' '.join(current_row))

# Create a DataFrame from the list of rows
df_table = pd.DataFrame(rows)

# Ensure DataFrame has columns and rename them if needed
# Assuming the first row of `rows` is the header
if not df_table.empty:
    header = df_table.iloc[0]
    df_table = df_table[1:]
    df_table.columns = header

# Save to Excel
df_table.to_excel("extracted_table_2308.xlsx", index=False)

print("Data extraction complete. Saved to extracted_table.xlsx")


Data extraction complete. Saved to extracted_table.xlsx


In [20]:
###working 

import pandas as pd
import re

# Load the previously extracted data from Excel
df_table = pd.read_excel("extracted_table_2308.xlsx", header=None)

# Function to clean and split lines where "O " is present
def clean_and_split(line):
    if isinstance(line, str) and line.startswith("O "):
        # Replace multiple spaces with a single space
        line = re.sub(r'\s+', ' ', line)
        # Split the line by spaces
        split_line = line.split(' ')
        # Join the elements with semicolons
        return ';'.join(split_line)
    return line

# Function to merge lines if a line might be a continuation of the previous line
def merge_lines(data):
    merged_data = []
    for i, line in enumerate(data):
        if i > 0:
            # Check if the current line is a continuation of the previous line
            if isinstance(line, str) and line.islower():
                # Merge with the previous line
                merged_data[-1] += " " + line.strip()
            else:
                merged_data.append(line)
        else:
            merged_data.append(line)
    return merged_data

# Apply the merge_lines function to combine split lines
merged_table = merge_lines(df_table[0].tolist())

# Apply the cleaning function to each row in the merged data
df_table_cleaned = pd.Series(merged_table).apply(clean_and_split)

# Split the semicolon-separated values into separate columns
df_table_split = df_table_cleaned.str.split(';', expand=True)

# Save the cleaned and split data to a new Excel file
df_table_split.to_excel("cleaned_extracted_table.xlsx", index=False)

print("Data cleaning and splitting complete. Saved to cleaned_extracted_table.xlsx")


Data cleaning and splitting complete. Saved to cleaned_extracted_table.xlsx


In [ ]:
##handles 
import pandas as pd
import re

# Load the previously extracted data from Excel
df_table = pd.read_excel("extracted_table.xlsx", header=None)

# Skip the first two rows and extract the remaining data
df_table_remaining = df_table.iloc[2:]

# Convert the DataFrame to a single string to process all rows together
all_text = ' '.join(df_table_remaining[0].dropna().astype(str).tolist())

# Split the text by "O " followed by a space, but keep the delimiter in the result
rows = re.split(r'(O )', all_text)
rows = [''.join(pair) for pair in zip(rows[1::2], rows[2::2])]

# Function to process each row, handling the required merging and splitting logic
def process_row(row):
    # Split the row into individual cells
    cleaned_matches = row.split()

    # Handle missing data between alphanumeric and numeric values
    final_row = []
    i = 0
    while i < len(cleaned_matches):
        if i + 2 < len(cleaned_matches) and re.match(r'^[A-Za-z0-9]+$', cleaned_matches[i]) and re.match(r'^\d{2}$', cleaned_matches[i + 1]) and '$' in cleaned_matches[i + 2]:
            # Add the current cell, the two-digit numeric cell, and the $ cell as separate cells
            final_row.append(cleaned_matches[i])
            final_row.append(cleaned_matches[i + 1])
            final_row.append(cleaned_matches[i + 2])
            i += 3
        else:
            final_row.append(cleaned_matches[i])
            i += 1

    return final_row

# Apply the processing to each row
processed_data = [process_row(row) for row in rows]

# Convert the processed data into a DataFrame
df_cleaned = pd.DataFrame(processed_data)

# Save the cleaned and split data to a new Excel file
df_cleaned.to_excel("cleaned_extracted_table_remaining_corrected.xlsx", index=False, header=False)

print("Data extraction and splitting complete. Saved to cleaned_extracted_table_remaining_corrected.xlsx")


In [ ]:
# ###  updated code handing words between alphanumeric and $ with _ and segment column 

import pandas as pd
import re

# Load the previously extracted data from Excel
df_table = pd.read_excel("extracted_table.xlsx", header=None)

# Function to clean and split lines where "O " is present
def clean_and_split(line):
    if isinstance(line, str) and line.startswith("O "):
        # Replace multiple spaces with a single space
        line = re.sub(r'\s+', ' ', line)
        # Split the line by spaces
        split_line = line.split(' ')

        # Logic 1: Add _ (underscore) between alphanumeric data and $ sign data
        for i in range(len(split_line) - 1):
            if re.match(r'^[A-Za-z0-9]+$', split_line[i]) and '$' in split_line[i + 1]:
                split_line[i] += "_" + split_line.pop(i + 1)
                break  # assuming only one such case per line

        # Logic 2: Handle provider, 2-digit numeric, and $ sign case
        if 'provider' in split_line:
            provider_idx = split_line.index('provider')
            if provider_idx + 2 < len(split_line) and re.match(r'^\d{2}$', split_line[provider_idx + 1]) and '$' in split_line[provider_idx + 2]:
                pass  # Leave the data as is, it fits the expected pattern
            else:
                # If the pattern doesn't match, insert 10 as the data after 'provider'
                split_line.insert(provider_idx + 1, '10')

        # Join the elements with semicolons
        return ';'.join(split_line)
    return line

# Function to merge lines if a line might be a continuation of the previous line
def merge_lines(data):
    merged_data = []
    for i, line in enumerate(data):
        if i > 0:
            # Check if the current line is a continuation of the previous line
            if isinstance(line, str) and line.islower():
                # Merge with the previous line
                merged_data[-1] += " " + line.strip()
            else:
                merged_data.append(line)
        else:
            merged_data.append(line)
    return merged_data

# Apply the merge_lines function to combine split lines
merged_table = merge_lines(df_table[0].tolist())

# Apply the cleaning function to each row in the merged data
df_table_cleaned = pd.Series(merged_table).apply(clean_and_split)

# Split the semicolon-separated values into separate columns
df_table_split = df_table_cleaned.str.split(';', expand=True)

# Save the cleaned and split data to a new Excel file
df_table_split.to_excel("cleaned_extracted_table.xlsx", index=False)

print("Data cleaning and splitting complete. Saved to cleaned_extracted_table.xlsx")
